# Machine Learning models on Mismatch Response data

In this notebook: 
- Necessary imports
- SVM model 
- Logistic Regression model
- Decision Tree model

## Imports

In [1]:
import os       # using operating system dependent functionality (folders)
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
import seaborn as sn
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '../eegyolk') # path to helper functions
import helper_functions as hf # library useful for eeg and erp data cleaning
import initialization_functions #library to import data
import epod_helper

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('df_avg_mmr.csv', sep = ',')

In [3]:
df

,2_Fp1_a,2_AF3_a,2_F7_a,2_F3_a,2_FC1_a,2_FC5_a,2_T7_a,2_C3_a,2_CP1_a,2_CP5_a,...,11_C4_b,11_T8_b,11_FC6_b,11_FC2_b,11_F4_b,11_F8_b,11_AF4_b,11_Fp2_b,11_Fz_b,11_Cz_b
0,3.981499,11.639081,-18.811622,18.343454,10.023632,-4.024143,-13.999364,-17.671889,-29.638338,-20.809440,...,-3.435515,1.738558,8.090447,13.989412,8.420473,23.518492,50.627080,33.244638,28.191358,40.034364
1,2.390371,-17.843979,-3.338771,7.536740,-10.998419,18.220947,16.625307,-4.156407,-17.935160,-6.441905,...,15.112951,36.641937,37.896288,19.602109,14.555350,41.145307,24.244979,58.820555,11.613820,9.621704
2,-18.065883,-17.785199,-12.858838,-8.709094,9.764843,-1.953743,-2.150446,11.090374,0.289517,2.440453,...,24.699454,32.055830,-4.361878,-5.785746,12.088077,9.451175,6.296128,-12.169311,0.450006,6.585673
3,52.196651,51.457712,22.673943,32.572107,5.180163,18.782386,20.845702,7.723524,5.208093,26.848718,...,52.046497,54.323672,58.501746,56.353772,49.522784,71.824112,42.726944,74.669478,39.794922,62.994571
4,-841.737108,-801.240011,-879.743229,-923.354318,-801.476465,-847.688256,-737.339387,-853.147914,-732.422316,-766.587103,...,-41.609304,-13.818254,-30.328267,-57.140936,-32.112574,-38.012191,-43.552506,-43.767460,-61.546375,-55.303023
5,1.179806,6.366317,-22.585827,-16.469787,-26.659719,-20.820073,-44.714848,-5.861646,-4.621675,-22.095876,...,-9.086049,-28.215221,23.216179,22.833120,18.658916,-4.978932,16.195296,15.155379,19.184594,8.517291
6,-104.473044,-114.661477,-115.992143,-104.786412,-97.066618,-104.369716,-116.949484,-105.987364,-99.901829,-111.842382,...,11.753093,17.640083,-9.471046,4.256348,3.366350,-6.142375,-9.982942,-23.373584,1.966171,3.573784
7,-36.590509,-35.888334,-32.303728,-41.009655,-39.813567,-35.966233,-32.699715,-45.066325,-43.378386,-42.693242,...,-12.600397,-3.867548,-47.104975,-9.607376,-15.133582,-21.744324,-5.319063,-5.319112,-7.363014,-18.200347
8,-13.833605,-3.938048,11.836307,-27.299734,-8.398938,-128.276705,25.861200,-4.607624,-26.808643,30.578071,...,-15.484524,-0.724520,2.189864,2.951773,3.218586,21.250784,-9.144113,-8.693288,5.496910,-12.589195
9,32.232969,33.826705,38.582741,10.956048,42.392605,35.946838,31.716506,53.214490,52.874939,28.465112,...,-21.800947,6.808585,-8.667056,-23.001667,-9.690261,18.572201,-42.575242,-48.704422,-22.459858,-52.576538


## Split data

In [107]:
y = df['Group_AccToParents'].values # dependant variable
#X = df.drop(['Group_AccToParents','ParticipantID'],axis=1).values   # independant features
#X = df[['Group_AccToParents','2_Fp1_a', '2_AF3_a', '2_F7_a', '2_F3_a', '2_FC1_a', '2_FC5_a', '2_Fz_a',
          '5_Fp1_a', '5_AF3_a','5_F7_a', '5_F3_a', '5_FC1_a', '5_FC5_a', '5_Fz_a', 
          '8_Fp1_a', '8_AF3_a', '8_F7_a', '8_F3_a', '8_FC1_a', '8_FC5_a', '8_Fz_a', 
          '11_Fp1_a', '11_AF3_a', '11_F7_a', '11_F3_a', '11_FC1_a', '11_FC5_a', '11_Fz_a', 
            'sex', 'age_months', '2_Fp1_b', '2_AF3_b', '2_F7_b', '2_F3_b', '2_FC1_b', '2_FC5_b','2_Fz_b', 
          '5_Fp1_b', '5_AF3_b', '5_F7_b', '5_F3_b', '5_FC1_b', '5_FC5_b','5_Fz_b',
          '8_Fp1_b', '8_AF3_b', '8_F7_b', '8_F3_b', '8_FC1_b', '8_FC5_b', '8_Fz_b', 
          '11_Fp1_b', '11_AF3_b', '11_F7_b', '11_F3_b', '11_FC1_b', '11_FC5_b', '11_Fz_b']]
X = df[['11_Fp1_a', '11_AF3_a', '11_F7_a', '11_F3_a', '11_FC1_a', '11_FC5_a', '11_Fz_a']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.40, random_state=42)

In [102]:
y

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0], dtype=int64)

## Scale data

In [103]:
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
X_test = sc.transform(X_test)

## SVM model

In [104]:
svm = SVC(kernel= 'linear', random_state=1, C=0.1)
svm.fit(X_train, y_train)

SVC(C=0.1, kernel='linear', random_state=1)

In [105]:
y_pred = svm.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

Accuracy: 0.667


In [106]:
y_pred

array([0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0], dtype=int64)

## Logistic Regression model